### D. Heslop, U. Amarathunga and E. J. Rohling (2023). North African Plio-Pleistocene monsoon runoff and its impacts on the Mediterranean Sea. Paleoceanography and Paleoclimatology (submitted).

#### Jupyter notebook to estimate box model sensitivities and in Figures 2 and 8. 

#### This notebook performs the calculation and the results can be plotted in ```Sensitivity plotter - GitHub.ipynb```

The Mediterranean box model is written in Julia (https://julialang.org/) and requires the model files:
1. ```rsl.forcing.jl```
2. ```rsl.utils.jl```
3. ```rsl.fixvar.jl```
4. ```rsl.constant.jl```
5. ```rsl.rndvar.jl```
6. ```rsl.estimate.jl```

which are available as part of this GitHub repository.

#### Read in Julia packages
These can be installed using the package manager (https://docs.julialang.org/en/v1/stdlib/Pkg/)

In [ ]:
using Statistics
using Random
using DelimitedFiles

#### Read in box model code as collection of files

In [ ]:
include("./rsl.forcing.jl");
include("./rsl.utils.jl");
include("./rsl.fixvar.jl");
include("./rsl.constant.jl");
include("./rsl.rndvar.jl");
include("./rsl.estimate.jl");

#### Seed the random number generator for repeatability

In [ ]:
Random.seed!(1234);

#### Generate box model constants (see ```rsl.constant.jl``` for definitions)

In [ ]:
C, ρa, AM, p = constants();

#### Generate box model fixed values (see ```rsl.fixvar.jl``` for definitions)

In [ ]:
A_p, SA_p, δA_p, P_p, X_p, R_p, δR_p, B_p, δB_p, M_p, δM_p, sinc_p = fixvar();

#### Set default box model parameters (which may change later)

In [ ]:
∇T_s = 5.0; #summer SST sensitivity to RSL_Gib
∇T_w = 3.5; #winter SST sensitivity to RSL_Gib
sl = 0; #RSL_Gib
T = 19.0; #Average SST
zmodel0 = 150.0; #model depth when monsoon is inactive
zmul0 = 30.0; #maximum thickness of monsoon upper layer
Δzmodel = -50.0; #change in model depth when monsoon is active

### Set fixed model parameters

In [ ]:
Ntrials = 500; #number of trials to assess uncertainties
Nyrs = 500; #number of years to run model over
Δt_s,Δt_m,Δt_w = 4/12, 2/12, 6/12; #duration of summer, monsoon, and winter periods
Mf_s,Mf_w = 0.0, 0.0; #No monsoon forcing in summer and winter
annual_forcing = true; #change random forcing each year
null_T = true; #use prescribed SST forcing
spinup = true; #start from a default state

#### Estimate sensitivity WRT $\delta^{18}$O composition of the monsoon FWF (i.e., $\partial\delta^{18}\mathrm{O_{pf}}$/$\partial\delta^{18}\mathrm{O_{mon}}$)

In [ ]:
δM_p0 = LinRange(-8,-12,11); #range of FWF d18O compositions to test
Mf_m = 4; #Set Delta FWF as 4
ΔT_m = 0; # No temperature concentration effect
sl0 = LinRange(-40,50,20) #array of RSL_Gib to consider   
pd = zeros(20,Ntrials) #initialize sensitivity output array

for j in 1:length(sl0) #loop through RSL_Gib values
    global sl = sl0[j] #Current RSL_Gib value
    global δcalcite_out = zeros(11,Ntrials) #output array for d18O of calcite
        for i in 1:length(δM_p0) #loop through monsoon compositions
        Random.seed!(1234); #reset seed to track individual trials (needed for later regression step)
        global δM_p = δM_p0[i]; #current monsoon d18O value
        global zmodel = zmodel0+Δzmodel; #adjust model depth
        global Δzmul = -3.509.*ΔT_m; #adjustment to monsoon layer depth
        global zmul = zmul0+Δzmul; #adjust monsoon layer depth
        δsml, Ssml, δssth, Sssth, δmul, Smul, δmll, Smll, δmsth, Smsth, δwml, Swml, T_w, T_s, T_m, δcalcite_w, δcalcite_s, δcalcite_m = estimate_years(Ntrials,Nyrs,annual_forcing); #sample model
        δcalcite_out[i,:] = δcalcite_m #store model output
    end
    
    for i in 1:Ntrials
        pd[j,i] = utils_linfit(δM_p0,δcalcite_out[:,i])[2] #assess sensitivity for each trial (gradient of first-order polynomial)
    end
end

pd_out = zeros(20,2)
for i in 1:20
    pd_out[i,1]=mean(pd[i,:]) #mean sensitivity for each RSL_Gib
    pd_out[i,2]=std(pd[i,:]) #standard error of sensitivity for each RSL_Gib
end

#Uncomment next two lines to save the output file
output = cat(-sl0,pd_out,dims=2)
writedlm("XMON.txt", output)

#### Estimate sensitivity WRT $\Delta$FWF (i.e., $\partial\delta^{18}\mathrm{O_{pf}}$/$\partial\Delta\mathrm{FWF}$)

In [ ]:
δM_p = -10; #set FWF d18O composition
Mf_m0 = LinRange(1,3,11); #range of DeltaFWF to test
ΔT_m = 0; #No temperature concentration effect
sl0 = LinRange(-40,50,20) #array of RSL_Gib to consider      
pd = zeros(20,Ntrials) #initialize sensitivity output array
for j in 1:length(sl0) #loop through RSL_Gib values
    global sl = sl0[j] #Current RSL_Gib value
    global δcalcite_out = zeros(11,Ntrials) #output array for d18O of calcite
        for i in 1:length(Mf_m0) #loop through DeltaFWF values
        Random.seed!(1234); #reset seed to track individual trials (needed for later regression step)
        global Mf_m = Mf_m0[i]; #current DeltaFWF value
        global zmodel = zmodel0+Δzmodel; #adjust model depth
        global Δzmul = -3.509.*ΔT_m; #adjustment to monsoon layer depth
        global zmul = zmul0+Δzmul; #adjust monsoon layer depth
        δsml, Ssml, δssth, Sssth, δmul, Smul, δmll, Smll, δmsth, Smsth, δwml, Swml, T_w, T_s, T_m, δcalcite_w, δcalcite_s, δcalcite_m = estimate_years(Ntrials,Nyrs,annual_forcing); #sample model
        δcalcite_out[i,:] = δcalcite_m #store model output
    end
    
    for i in 1:Ntrials
        pd[j,i] = utils_linfit(Mf_m0,δcalcite_out[:,i])[2] #assess sensitivity for each trial (gradient of first-order polynomial)
    end
end

pd_out = zeros(20,2)
for i in 1:20
    pd_out[i,1]=mean(pd[i,:]) #mean sensitivity for each RSL_Gib
    pd_out[i,2]=std(pd[i,:]) #standard error of sensitivity for each RSL_Gib
end

#Uncomment next two lines to save the output file
output = cat(-sl0,pd_out,dims=2)
writedlm("XFWF.txt", output)

#### Estimate sensitivity WRT $\Delta$T (i.e., $\partial\delta^{18}\mathrm{O_{pf}}$/$\partial\Delta\mathrm{T}$)

In [ ]:
δM_p = -10; #set FWF d18O composition
Mf_m = 0; #set DeltaFWF to zero
ΔT_m0 = LinRange(0,3,11); #range of DeltaT to test
sl0 = LinRange(-40,50,20)  #array of RSL_Gib to consider   
pd = zeros(20,Ntrials) #initialize sensitivity output array
for j in 1:length(sl0) #loop through RSL_Gib values
    global sl = sl0[j] #Current RSL_Gib value
    global δcalcite_out = zeros(11,Ntrials) #output array for d18O of calcite
        for i in 1:length(ΔT_m0) #loop through DeltaT values
        Random.seed!(1234); #reset seed to track individual trials (needed for later regression step)
        global ΔT_m = ΔT_m0[i]; #current DeltaT value
        global zmodel = zmodel0+Δzmodel; #adjust model depth
        global Δzmul = -3.509.*ΔT_m; #adjustment to monsoon layer depth
        global zmul = zmul0+Δzmul; #adjust monsoon layer depth
        δsml, Ssml, δssth, Sssth, δmul, Smul, δmll, Smll, δmsth, Smsth, δwml, Swml, T_w, T_s, T_m, δcalcite_w, δcalcite_s, δcalcite_m = estimate_years(Ntrials,Nyrs,annual_forcing); #sample model
        δcalcite_out[i,:] = δcalcite_m #store model output
    end
    
    for i in 1:Ntrials
        pd[j,i] = utils_linfit(ΔT_m0,δcalcite_out[:,i])[2]
    end
end

pd_out = zeros(20,2)
for i in 1:20
    pd_out[i,1]=mean(pd[i,:]) #mean sensitivity for each RSL_Gib
    pd_out[i,2]=std(pd[i,:]) #standard error of sensitivity for each RSL_Gib
end

#Uncomment next two lines to save the output file
output = cat(-sl0,pd_out,dims=2)
writedlm("XT.txt", output)

#### Estimate sensitivity WRT $\mathrm{RSL_{Gib}}$ (i.e., $\partial\delta^{18}\mathrm{O_{pf}}$/$\partial\mathrm{RSL_{Gib}}$)

In [ ]:
Mf_m = 0; #set DeltaFWF to zero
sl0 = LinRange(-40,50,20) #array of RSL_Gib to consider 
ΔT_m = 0; #No temperature concentration effect
Δt_s,Δt_m,Δt_w = 6/12, 0/12, 6/12; #switch off monsoon

global δcalcite_out = zeros(20,Ntrials) #output array for d18O of calcite

for i in 1:length(sl0) #loop through RSL_Gib values
    Random.seed!(1234); #reset seed to track individual trials (needed for later regression step)
    global sl = sl0[i]; #current RSL_Gib value
    global zmodel = zmodel0; #model depth
    δsml, Ssml, δssth, Sssth, δmul, Smul, δmll, Smll, δmsth, Smsth, δwml, Swml, T_w, T_s, T_m, δcalcite_w, δcalcite_s, δcalcite_m = estimate_years(Ntrials,Nyrs,annual_forcing); #sample model
    δcalcite_out[i,:] = δcalcite_s #store model output
end

pd = zeros(20,Ntrials)
for i in 1:Ntrials
    pp = utils_quadfit(sl0,δcalcite_out[:,i]) #fit quadratic (see Rohling et al. 2014)
    pd[:,i] = pp[2].+2.0.*pp[3].*sl0 #derivative of quadratic
end

pd_out = zeros(20,2)
for i in 1:20
    pd_out[i,1]=mean(pd[i,:]) #mean sensitivity for each RSL_Gib
    pd_out[i,2]=std(pd[i,:]) #standard error of sensitivity for each RSL_Gib
end

#Uncomment next two lines to save the output file
output = cat(-sl0,-pd_out,dims=2)
writedlm("XRSL.txt", output)